<a href="https://colab.research.google.com/github/masoudcharkhabi/ML-from-Expert-Preferences/blob/abstractions/colab_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

### Packages

In [1]:
from google.colab import drive
import os
from getpass import getpass
import wandb
import os
import wandb
import datetime
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Disable TensorFlow info, warning, and error messages.


### Huggingface login

In [2]:
# Prompt for Hugging Face API token without saving it
huggingface_token = getpass("Please enter your Hugging Face API token: ")

# Set the token as an environment variable
os.environ["HUGGINGFACE_TOKEN"] = huggingface_token

# Log in using the token
from huggingface_hub import login
login(token=os.environ["HUGGINGFACE_TOKEN"])

Please enter your Hugging Face API token: ··········


### Make sure you have a GPU and High memory

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Wed Dec  4 06:40:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   55C    P8              17W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Clone Github repo (you need private-public keys with the same naming convention)

In [5]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
# Create symbolic links to the SSH keys in Drive
!ln -s /content/drive/MyDrive/ssh_keys/id_colab ~/.ssh/id_colab
!ln -s /content/drive/MyDrive/ssh_keys/id_colab.pub ~/.ssh/id_colab.pub

# Set the correct permissions for SSH keys
!chmod 600 ~/.ssh/id_colab
!chmod 644 ~/.ssh/id_colab.pub

# Start SSH agent and add key
!eval "$(ssh-agent -s)"
!ssh-add ~/.ssh/id_colab

# Create SSH config
ssh_config = """
Host github.com
  HostName github.com
  User git
  IdentityFile ~/.ssh/id_colab
  StrictHostKeyChecking no
"""

# Create the .ssh directory if it doesn't exist
os.makedirs(os.path.expanduser("~/.ssh"), exist_ok=True)

with open(os.path.expanduser("~/.ssh/config"), "w") as f:
    f.write(ssh_config)

ln: failed to create symbolic link '/root/.ssh/id_colab': File exists
ln: failed to create symbolic link '/root/.ssh/id_colab.pub': File exists
Agent pid 7413
Could not open a connection to your authentication agent.


In [7]:
# Test SSH connection
!ssh -T git@github.com

Hi masoudcharkhabi! You've successfully authenticated, but GitHub does not provide shell access.


In [8]:
repo_ssh_url = "git@github.com:masoudcharkhabi/ML-from-Expert-Preferences.git"
branch_name = "abstractions"
!git clone -b {branch_name} --single-branch {repo_ssh_url}

fatal: destination path 'ML-from-Expert-Preferences' already exists and is not an empty directory.


In [9]:
repo_name = "ML-from-Expert-Preferences"
os.chdir(repo_name)

In [10]:
!git branch
!ls -ltr

* abstractions
total 104
-rw-r--r-- 1 root root  2680 Dec  4 06:21 README.md
drwxr-xr-x 3 root root  4096 Dec  4 06:21 baseline
-rw-r--r-- 1 root root 55590 Dec  4 06:21 colab_experiments.ipynb
drwxr-xr-x 4 root root  4096 Dec  4 06:21 cs329h-project
-rw-r--r-- 1 root root  2053 Dec  4 06:21 train.py
-rw-r--r-- 1 root root   204 Dec  4 06:21 requirements.txt
-rw-r--r-- 1 root root  3515 Dec  4 06:21 eval.py
-rw-r--r-- 1 root root  2001 Dec  4 06:21 data_prep.py
drwxr-xr-x 5 root root  4096 Dec  4 06:21 data
drwxr-xr-x 3 root root  4096 Dec  4 06:25 wandb
drwxr-xr-x 3 root root  4096 Dec  4 06:25 models
drwxr-xr-x 2 root root  4096 Dec  4 06:28 __pycache__
-rw-r--r-- 1 root root  1035 Dec  4 06:37 serve.py


In [31]:
!pip install -r requirements.txt

#TODO figure out the verions and added to requirements later
!pip install datasets
!pip install torch
!pip install evaluate
!pip install rouge_score

  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached ray-2.40.0-cp310-cp310-manylinux2014_x86_64.whl.metadata (17 kB)
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b1dd237788da12452af2e8d6dffe651219ed4f85515a4c4f29f059c5f4f82ea9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


# Data prep

In [13]:
from transformers import AutoTokenizer
from data_prep import DataPreparation

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # Replace "t5-small" with the appropriate model name

# Create instance of DataPreparation
data_preparation = DataPreparation(dataset_path="ai2-adapt-dev/flan_v2_converted")

# Load the dataset
dataset = data_preparation.load_data()

# Prepare the dataset (tokenize)
train_dataset = data_preparation.prepare_dataset(tokenizer=tokenizer)

# Now train_dataset is ready for training
# Get dataset information
data_preparation.dataset_info()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


Split: train, Number of examples: 89982


# Fine-tune

### This will require and Weights and Biases API key for logging

In [14]:
# Import necessary classes
from train import ModelTrainer

# Step 1: Load tokenizer
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 2: Load and prepare the dataset
data_preparation = DataPreparation(dataset_path="ai2-adapt-dev/flan_v2_converted")
dataset = data_preparation.load_data()
train_dataset = data_preparation.prepare_dataset(tokenizer=tokenizer)

# Step 3: Initialize ModelTrainer and setup training
trainer = ModelTrainer(model_name=model_name)
trainer.setup_training(train_dataset=train_dataset, tokenizer=tokenizer)

# Step 4: Train the model
trainer.train_model()


Repo card metadata block was not found. Setting CardData to empty.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mcharkhabi (ai-eval). Use `wandb login --relogin` to force relogin


/content/ML-from-Expert-Preferences/train.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  self.trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


Model saved to: ./models/fine_tuned_model_20241204_064440


train/epoch,▁
train/global_step,▁
total_flos,6225722867712.0
train/epoch,0.00051
train/global_step,23
train_loss,18.22854
train_runtime,4.9362
train_samples_per_second,9.114
train_steps_per_second,4.659


# Serve

In [27]:
# from serve import ModelServer

# Step 1: Load the tokenizer
model_name = "t5-small"  # TODO: Replace with fine-tuned model later
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 2: Initialize ModelServer
model_server = ModelServer(model_name=model_name)

# Step 3: Set the tokenizer for the model server
model_server.set_tokenizer(tokenizer)

# Step 4: Run inference
input_text = "Translate the following sentence to French: 'The weather is nice today.'"
output_text = model_server.run_inference(input_text)

# Step 5: Print the result
print("Generated Output:", output_text)

# Optional: Store the output in a file
experiment_id = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_path = "./data/output/"+experiment_id+"_output.txt"
model_server.store_output(input_text, output_path)
print(f"Output stored in: {output_path}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated Output: « Le temps est agréable aujourd'hui ».
Output stored in: ./data/output/20241204_065255_output.txt


# Eval

In [32]:
from eval import ModelEvaluator

# Step 1: Load the tokenizer
model_name = "t5-small"

# TODO: Replace with  fine-tuned model path (including the correct timestamp) later
# model_name = "./fine_tuned_model_20231124_123456"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 2: Create instance of DataPreparation and load the dataset
data_preparation = DataPreparation(dataset_path="ai2-adapt-dev/flan_v2_converted")
dataset = data_preparation.load_data()

# Step 3: Prepare evaluation dataset
eval_dataset = data_preparation.prepare_dataset(tokenizer=tokenizer)

# Step 4: Initialize ModelEvaluator
evaluator = ModelEvaluator(model_name=model_name, tokenizer=tokenizer)

# Step 5: Run evaluation
evaluation_results = evaluator.evaluate(eval_dataset)

# Step 6: Print the evaluation results
print("Evaluation Results:", evaluation_results)


Repo card metadata block was not found. Setting CardData to empty.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


accuracy,▁
f1_score,▁
perplexity,▁
precision,▁
recall,▁
accuracy,0
f1_score,0
perplexity,76779512.0
precision,0
recall,0


Evaluation Results: {'accuracy': 0.0, 'f1_score': 0.0, 'precision': 0.0, 'recall': 0.0, 'bleu': {'bleu': 0.03550836856172761, 'precisions': [0.11895388076490439, 0.03905120046282904, 0.023435182438445563, 0.014602981442044418], 'brevity_penalty': 1.0, 'length_ratio': 1.2912127814088599, 'translation_length': 3556, 'reference_length': 2754}, 'rouge': {'rouge1': 0.10575423600418363, 'rouge2': 0.03249340922976353, 'rougeL': 0.0822570979963348, 'rougeLsum': 0.08913628656954073}, 'perplexity': 76779512.0}
